In [26]:
%pip install fuzzywuzzy spacy
!python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 12.8/12.8 MB 1.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from fuzzywuzzy import fuzz
import spacy
from difflib import SequenceMatcher as SM

c:\Projects\service-document-elasticsearch\araks_venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [27]:
nlp = spacy.load("en_core_web_sm")

In [2]:
async def fuzzy_similar(a, b):
    return (fuzz.token_sort_ratio(a, b))

In [3]:
await fuzzy_similar('', "")

100

In [3]:
async def exact_similar(a, b):
    return (sorted(a.lower().split()) == sorted(b.lower().split()))

In [210]:
source_props = ["Danoyan", "Yerevan 15 Armenia"]
target_props = ["15 Yerevan Armenia", "Armen Danoyan"]

In [211]:
nlp(source_props[0]).similarity(nlp(target_props[1]))

0.750638044548437

In [9]:
def list_to_str(item):
    if type(item) == list and len(item) == 1 and item[0]:
            return str(item[0])
    else: return str(item)

In [4]:
async def match(source, target, method):
        if method == "fuzzy":
                    if type(target[1]) == list and len(target[1]) > 1:
                        ratio = max([await fuzzy_similar(list_to_str(source[1]), trg) for trg in target[1]])
                    else:
                        ratio = await fuzzy_similar(list_to_str(source[1]), target[1])
                    if ratio >= 20:
                        print(ratio)
                        return source[0], target[0]
                        
        elif method == "exact":
                    if type(target[1]) == list and len(target[1]) > 1:
                        ratio = any([await exact_similar(list_to_str(source[1]), trg) for trg in target[1]])
                    else:
                        ratio = await exact_similar(list_to_str(source[1]), target[1])
                    if ratio:
                        return source[0], target[0]

In [193]:
await match((12333, "Armenia"), (124412515, ["ArmeNia", "RUSIA"]), "exact")

(12333, 124412515)

In [112]:
data = [{"source" : [{"name" : "Yerevan 15 Armenia", "id" : 0}], "target" : [{"name" : "15 Yerevan Armenia", "id" : 1}], "method" : "fuzzy"}, {"source" : [{"name" : "Danoyan Armen", "id" : 2}], "target" : [{"name" : "Armen   Danoyan", "id" : 3}], "method" : "exact"}]

In [113]:
import asyncio

async def process_relations(rel_data):
    
    matched_ids = []
    
    for rel in rel_data:
        sources = rel["source"]
        targets = rel["target"]
        method = rel["method"]
        
        tasks = [match(source, target, method) for source in sources for target in targets]
        results = await asyncio.gather(*tasks)
        
        matched_ids.extend(results)
    
    return matched_ids

In [ ]:
await process_relations(data)

In [3]:
%pip install psycopg2

     ---------------------------------------- 1.2/1.2 MB 3.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Projects\\service-document-elasticsearch\\araks_venv\\Lib\\site-packages\\psycopg2\\_psycopg.cp310-win_amd64.pyd'
Check the permissions.


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
{
  "source_id": "44f4b7e5-28f4-4023-879f-8d913ea15b74",
  "target_id": "b614a89f-a12d-45d7-9e13-32fce423172f",
  "source_props": ["b614a89f-a12d-45d7-9e13-32fce423172f", "b614a89f-a12d-45d7-9e13-32fce423172f"],
  "source_prop_default": "b614a89f-a12d-45d7-9e13-32fce423172f",
  "target_props": ["b614a89f-a12d-45d7-9e13-32fce423172f", "b614a89f-a12d-45d7-9e13-32fce423172f"],
  "target_prop_default": "b614a89f-a12d-45d7-9e13-32fce423172f",
  "methods": ["exact", "fuzzy", "fuzzy"]
}

In [5]:
import psycopg2 as ps
import os
import asyncio

In [6]:
type_ids = ['5d107f9f-f8cf-427e-9fd7-52b92bf40ce4', '5d107f9f-f8cf-427e-9fd7-52b92bf40ce4']
pntp_ids = {"source" : ['7e60664d-0f3b-4a5f-9ac8-5319ee49730f'], "target" : ['181536e0-3d75-44f7-8b61-60c43b105b97'], "methods" : ["fuzzy"]}
source_default_prop = '7e60664d-0f3b-4a5f-9ac8-5319ee49730f'
target_default_prop = '7e60664d-0f3b-4a5f-9ac8-5319ee49730f'

In [7]:
select_deafault_script = """SELECT
                        np.node_id AS node_id,
                        -- pntp.ref_property_type_id AS type,
                        -- pntp.name AS key,
                        np.nodes_data AS value
                        FROM projects_node_type_property AS pntp
                        LEFT JOIN(SELECT id, node_id, project_type_property_id, nodes_data FROM node_properties) AS np ON np.project_type_property_id = pntp.id
                        WHERE pntp.project_type_id = '{type_id}'
                        AND pntp.id = '{property_id}'
                        """

select_props_script = "SELECT id AS node_id, name AS value from nodes where project_type_id = '{type_id}'"

In [10]:
datas= {}
results = []
tasks = []
conn = ps.connect(dbname="araks", user="postgres",
                  password="tik555", port="5433")

for i in range(len(pntp_ids['source'])):
    
    cur = conn.cursor()
    
    if pntp_ids['source'][i] not in datas:
        if source_default_prop == pntp_ids['source'][i]:
            cur.execute(select_props_script.format(type_id = type_ids[0]))

        else:
            cur.execute(select_deafault_script.format(type_id = type_ids[0], property_id = pntp_ids['source'][i]))

        datas[pntp_ids['source'][i]] = cur.fetchall()

        cur.close()
        
    cur = conn.cursor()

    if pntp_ids['target'][i] not in datas:
        if target_default_prop == pntp_ids['target'][i]:
             cur.execute(select_props_script.format(type_id = type_ids[1]))

        else:
            cur.execute(select_deafault_script.format(type_id = type_ids[1], property_id = pntp_ids['target'][i]))
        
        datas[pntp_ids['target'][i]] = cur.fetchall()

        cur.close()

    for j, src_item in enumerate(datas[pntp_ids['source'][i]]):
        if pntp_ids['source'][i] == pntp_ids['target'][i]:
            for trg_item in datas[pntp_ids['target'][i]][j:]:
                if src_item[0] != trg_item[0] and (src_item[0], trg_item[0]) not in results and (trg_item[0], src_item[0]) not in results:
                    result = tasks.append(await match(src_item, trg_item, pntp_ids['methods'][i]))
                    if result:
                        results.append(result)
        else: 
            for trg_item in datas[pntp_ids['target'][i]]:
                if src_item[0] != trg_item[0] and (src_item[0], trg_item[0]) not in results and (trg_item[0], src_item[0]) not in results:
                    result = await match(src_item, trg_item, pntp_ids['methods'][i])
                    if result:
                        results.append(result)

conn.close()
datas.clear()
results

25
20
24
21
29
29
24
24
24
25


[('3eeb0142-dee7-4b56-88fe-856ccc96fa55',
  'c170d160-9af5-4f2f-bd83-6b1dc3f6807c'),
 ('1a82b174-5c79-4fdf-b63c-73525e442d40',
  'c170d160-9af5-4f2f-bd83-6b1dc3f6807c'),
 ('6a4b67c2-f425-4a4f-a466-548235bbd6bd',
  '3eeb0142-dee7-4b56-88fe-856ccc96fa55'),
 ('d3588e95-bd20-4037-a65d-f0f9fec84a53',
  'c170d160-9af5-4f2f-bd83-6b1dc3f6807c'),
 ('d3588e95-bd20-4037-a65d-f0f9fec84a53',
  'b2ffb875-4657-409c-bd14-0082fb318bfa'),
 ('d3588e95-bd20-4037-a65d-f0f9fec84a53',
  '3eeb0142-dee7-4b56-88fe-856ccc96fa55'),
 ('2e24a8cc-396a-47f0-b98f-0b58db9db00a',
  'b2ffb875-4657-409c-bd14-0082fb318bfa'),
 ('9fe24d1c-883a-472a-a970-f05adbb59841',
  'b2ffb875-4657-409c-bd14-0082fb318bfa'),
 ('9fe24d1c-883a-472a-a970-f05adbb59841',
  '3eeb0142-dee7-4b56-88fe-856ccc96fa55'),
 ('b2ffb875-4657-409c-bd14-0082fb318bfa',
  '6a4b67c2-f425-4a4f-a466-548235bbd6bd')]

In [11]:
datas= {}
tasks = []
conn = ps.connect(dbname="araks", user="postgres",
                  password="tik555", port="5433")

for 

               
# for i in range(len(pntp_ids['source'])):
    
#     cur = conn.cursor()
    
#     if pntp_ids['source'][i] not in datas:
#         if source_default_prop == pntp_ids['source'][i]:
#             cur.execute(select_props_script.format(type_id = type_ids[0]))

#         else:
#             cur.execute(select_deafault_script.format(type_id = type_ids[0], property_id = pntp_ids['source'][i]))

#         datas[pntp_ids['source'][i]] = cur.fetchall()

#         cur.close()
        
#     cur = conn.cursor()

#     if pntp_ids['target'][i] not in datas:
#         if target_default_prop == pntp_ids['target'][i]:
#              cur.execute(select_props_script.format(type_id = type_ids[1]))

#         else:
#             cur.execute(select_deafault_script.format(type_id = type_ids[1], property_id = pntp_ids['target'][i]))
        
#         datas[pntp_ids['target'][i]] = cur.fetchall()

#         cur.close()

#     for j, src_item in enumerate(datas[pntp_ids['source'][i]]):
#         if pntp_ids['source'][i] == pntp_ids['target'][i]:
#             for trg_item in datas[pntp_ids['target'][i]][j:]:
#                 if src_item[0] != trg_item[0]:
#                     tasks.append(match(src_item, trg_item, pntp_ids['methods'][i]))
#                     # if result:
#                     #     results.append(result)
#         else: 
#             for trg_item in datas[pntp_ids['target'][i]]:
#                 if src_item[0] != trg_item[0]:
#                     tasks.append(match(src_item, trg_item, pntp_ids['methods'][i]))
#                     # if result:
#                     #     results.append(result)

# conn.close()
# datas.clear()
# results = [value for value in await asyncio.gather(*tasks) if value is not None]
results
# await remove_duplicates(results)

25
36
20
24
21
29
29
24
24
24
25
25
25
22


[('3eeb0142-dee7-4b56-88fe-856ccc96fa55',
  'c170d160-9af5-4f2f-bd83-6b1dc3f6807c'),
 ('c170d160-9af5-4f2f-bd83-6b1dc3f6807c',
  '3eeb0142-dee7-4b56-88fe-856ccc96fa55'),
 ('1a82b174-5c79-4fdf-b63c-73525e442d40',
  'c170d160-9af5-4f2f-bd83-6b1dc3f6807c'),
 ('6a4b67c2-f425-4a4f-a466-548235bbd6bd',
  '3eeb0142-dee7-4b56-88fe-856ccc96fa55'),
 ('d3588e95-bd20-4037-a65d-f0f9fec84a53',
  'c170d160-9af5-4f2f-bd83-6b1dc3f6807c'),
 ('d3588e95-bd20-4037-a65d-f0f9fec84a53',
  'b2ffb875-4657-409c-bd14-0082fb318bfa'),
 ('d3588e95-bd20-4037-a65d-f0f9fec84a53',
  '3eeb0142-dee7-4b56-88fe-856ccc96fa55'),
 ('2e24a8cc-396a-47f0-b98f-0b58db9db00a',
  'b2ffb875-4657-409c-bd14-0082fb318bfa'),
 ('9fe24d1c-883a-472a-a970-f05adbb59841',
  'b2ffb875-4657-409c-bd14-0082fb318bfa'),
 ('9fe24d1c-883a-472a-a970-f05adbb59841',
  '3eeb0142-dee7-4b56-88fe-856ccc96fa55'),
 ('b2ffb875-4657-409c-bd14-0082fb318bfa',
  '6a4b67c2-f425-4a4f-a466-548235bbd6bd'),
 ('b2ffb875-4657-409c-bd14-0082fb318bfa',
  'd3588e95-bd20-4037-a

In [40]:
async def remove_duplicates(input_list):
    seen = set()
    unique_list = []

    for item in input_list:
        frozen_item = frozenset(item.items())

        if frozen_item not in seen:
            seen.add(frozen_item)
            unique_list.append(item)

    return unique_list

In [144]:
output = [{'node_id' : item[0], 'prop_id' : item[1], 'prop_names' : item[2]} for node in datas for item in node]

In [145]:
output

[{'node_id': '3eeb0142-dee7-4b56-88fe-856ccc96fa55',
  'prop_id': '7e60664d-0f3b-4a5f-9ac8-5319ee49730f',
  'prop_names': 'Anna'},
 {'node_id': 'c170d160-9af5-4f2f-bd83-6b1dc3f6807c',
  'prop_id': '7e60664d-0f3b-4a5f-9ac8-5319ee49730f',
  'prop_names': 'Karen'},
 {'node_id': '1a82b174-5c79-4fdf-b63c-73525e442d40',
  'prop_id': '7e60664d-0f3b-4a5f-9ac8-5319ee49730f',
  'prop_names': 'Document Management Sector'},
 {'node_id': '6a4b67c2-f425-4a4f-a466-548235bbd6bd',
  'prop_id': '7e60664d-0f3b-4a5f-9ac8-5319ee49730f',
  'prop_names': 'Saved Graph Queries'},
 {'node_id': 'd3588e95-bd20-4037-a65d-f0f9fec84a53',
  'prop_id': '7e60664d-0f3b-4a5f-9ac8-5319ee49730f',
  'prop_names': 'Alerting System'},
 {'node_id': '2e24a8cc-396a-47f0-b98f-0b58db9db00a',
  'prop_id': '7e60664d-0f3b-4a5f-9ac8-5319ee49730f',
  'prop_names': 'Node-Based Application Forms'},
 {'node_id': '9fe24d1c-883a-472a-a970-f05adbb59841',
  'prop_id': '7e60664d-0f3b-4a5f-9ac8-5319ee49730f',
  'prop_names': 'Property-Based Cal

In [154]:
for i in range(len(pntp_ids['source'])):
    for src_item in [{'name' : out['prop_names'], 'id' : out['node_id']} if out['prop_id'] == pntp_ids['source'][i] for out in output]:
        for out_item in [{'name' : out['prop_names'], 'id' : out['node_id']} if out['prop_id'] == pntp_ids['target'][i] for out in output]:
            print(await match(src_item, out_item, pntp_ids['methods'][i]))

SyntaxError: expected 'else' after 'if' expression (2940355803.py, line 2)

In [11]:
my_list = [23, 44, 21]
all(isinstance(x, int) and 21 <= x <= 100 for x in my_list)

False